In [1]:
# get the words those are needed to be skipped
stop_words_file = open("F:\GoogleDrive-2\DS Courses\Applied Data Science in Python\Text Mining\My Codes\stop_words.txt", "r")
stop_words = stop_words_file.read().replace('\n', '').replace('"', '').split(',')
stop_words

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [2]:
import pickle
import gensim
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
# Load the list of documents
with open('newsgroups', 'rb') as f:
    newsgroup_data = pickle.load(f)

In [4]:
newsgroup_data[0]

"The best group to keep you informed is the Crohn's and Colitis Foundation\nof America.  I do not know if the UK has a similar organization.  The\naddress of\nthe CCFA is \n\nCCFA\n444 Park Avenue South\n11th Floor\nNew York, NY  10016-7374\nUSA\n\nThey have a lot of information available and have a number of newsletters.\n \nGood Luck."

In [5]:
# Use CountVectorizor to find three letter or more tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
#vect = CountVectorizer(min_df=20, max_df=0.2, stop_words=frozenset(stop_words), 
                      #token_pattern='(?u)\\b\\w\\w\\w+\\b')

vect = CountVectorizer(min_df=20, max_df=0.2, stop_words= frozenset(stop_words), 
                       token_pattern='[A-Za-z]{2,}')

In [6]:
# Fit and transform
X = vect.fit_transform(newsgroup_data)
X

C:\Users\User\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['mon'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


<2000x749 sparse matrix of type '<class 'numpy.int64'>'
	with 29022 stored elements in Compressed Sparse Row format>

In [7]:
# Convert sparse matrix to gensim corpus.
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Mapping from word IDs to words (To be used in LdaModel's id2word parameter)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())

In [8]:
# Use the gensim.models.ldamodel.LdaModel constructor to estimate 
# LDA model parameters on the corpus, and save to the variable `ldamodel`
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=20, id2word=id_map, passes=25, random_state=34)

In [9]:
# Using ldamodel, find a list of the 10 topics and the most significant 10 words in each topic. 
# those numbers are the relative importance of each word in the topic. The reason they don't add upto 1 
# is because by default print_topics show 10. 
# If you show 100 or so the sum will start getting close to 1
ldamodel.print_topics(20, num_words=10)

[(0,
  '0.017*"cs" + 0.016*"side" + 0.015*"post" + 0.013*"time" + 0.013*"net" + 0.012*"doctor" + 0.011*"weeks" + 0.011*"uk" + 0.011*"ac" + 0.011*"men"'),
 (1,
  '0.027*"people" + 0.025*"problems" + 0.024*"life" + 0.018*"moon" + 0.018*"idea" + 0.017*"long" + 0.014*"good" + 0.013*"great" + 0.013*"question" + 0.013*"break"'),
 (2,
  '0.043*"ground" + 0.036*"current" + 0.020*"good" + 0.017*"circuit" + 0.014*"hope" + 0.014*"year" + 0.012*"output" + 0.012*"high" + 0.012*"noise" + 0.012*"turn"'),
 (3,
  '0.028*"good" + 0.026*"mhz" + 0.023*"chip" + 0.021*"speed" + 0.020*"work" + 0.017*"auto" + 0.017*"switch" + 0.016*"find" + 0.015*"manual" + 0.015*"motherboard"'),
 (4,
  '0.025*"car" + 0.024*"system" + 0.022*"time" + 0.020*"work" + 0.016*"oil" + 0.013*"problem" + 0.011*"dos" + 0.011*"port" + 0.011*"info" + 0.010*"type"'),
 (5,
  '0.059*"god" + 0.039*"people" + 0.025*"argument" + 0.020*"evidence" + 0.019*"atheism" + 0.019*"true" + 0.018*"religion" + 0.014*"posting" + 0.012*"subject" + 0.012*"re

In [10]:
new_doc = ["\n\nIt's my understanding that the freezing will start to occur because old box use did much damage \
of the\ngrowing distance of Pluto and Charon from the Sun, due to it's\nelliptical orbit. \
It is not due to shadowing effects. \n\n\nPluto can shadow Charon, and vice-versa.\n\nGeorge \
Krumins\n-- "]

In [11]:
new_doc1=["5G is a wireless technology offering an enormous increase in transmission bandwidth, around 10 gigabits per second (GBit/s). Hence, downloading large data in a few seconds is directly pointing to Big Data. This will cause tremendous mobile traffic which may affect the network performance badly. Here comes the need for Cloud Computing technologies to be integrated with mobile systems to help in facilitating computing intensive services."]

In [12]:
new_doc2=["The government mulls over increasing the cash incentive for exports by one percentage point as it looks to motivate exporters to leverage the sudden opportunities presented by the US-China trade war. Currently, 26 sectors are provided with cash incentives ranging from 2 percent to 20 percent of their export"]

In [13]:
new_doc3=["The team were accorded a reception by the students as they entered the premises, before making their way down a slope that led to an expansive field with three centre wickets. Mashrafe, who arrived in Cardiff on Thursday night after attending the official captains’ press conference in the afternoon, was the only one of the 15 who did not physically take part in the practice but the inspirational leader was seen discussing strategy with the coach and later cracking jokes with teammates on the sidelines."]

In [14]:
new_doc4=["China berated the United States for  “bullying” Huawei on Thursday as Panasonic joined a parade of foreign companies reviewing their ties with the telecom giant after a US ban linked to security concerns."]

In [15]:
def topic_distribution(doc_to_be_tested):
    
    # Your Code Here
    
     # Fit and transform
    X = vect.transform(doc_to_be_tested)

    # Convert sparse matrix to gensim corpus.
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

    output = list(ldamodel[corpus])[0]
    
    return (output) 

In [19]:
topic_distribution(new_doc1)

[(3, 0.38621557), (7, 0.5319615)]